In [73]:
import json
import os

import numpy as np

STRATEGIES = ['only_spot', 'strawman', 'pair_amortize', 'ideal_ilp_overhead', 'on_demand']
PREEMPTION_RATES = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]

simulate_results = {}
def init_dict(d):
    for strategy in STRATEGIES:
        d[strategy] = {}
        for preemption_rate in PREEMPTION_RATES:
            d[strategy][preemption_rate] = None
init_dict(simulate_results)

# Load the results
for file in os.listdir('../exp'):
    if 'hourly_rate' not in file:
        continue
    with open(os.path.join('../exp', file), 'r') as f:
        results = json.load(f)
    strategy = file.split('-')[0].split('_avg_')[0]
    rate = float(file.split('hourly_rate_')[1].split('-')[0])
    simulate_results[strategy][rate] = np.array(results['costs'])
# print(simulate_results)
for strategy in STRATEGIES:
    for preemption_rate in PREEMPTION_RATES:
        if simulate_results[strategy][preemption_rate] is None:
            print(strategy, preemption_rate)

for strategy in STRATEGIES:
    if strategy == 'on_demand':
        continue
    for preemption_rate in PREEMPTION_RATES:
        if simulate_results[strategy][preemption_rate] is None:
            continue
        simulate_results[strategy][preemption_rate] = simulate_results[strategy][preemption_rate] / simulate_results['on_demand'][preemption_rate]

print(simulate_results['strawman'])


ideal_ilp_overhead 0.1
{0.1: array([0.32240801, 0.32240801, 0.32350838, 0.32350838, 0.32350838,
       0.32350838, 0.32130764, 0.32130764, 0.32350838, 0.32240801,
       0.32350838, 0.32020727, 0.32680949, 0.32240801, 0.32240801,
       0.32350838, 0.32350838, 0.32130764, 0.32020727, 0.32130764,
       0.32350838, 0.32240801, 0.32130764, 0.32130764, 0.32130764,
       0.32130764, 0.32130764, 0.32020727, 0.32130764, 0.32240801,
       0.32020727, 0.32130764, 0.32130764, 0.32460875, 0.32460875,
       0.32350838, 0.32130764, 0.32130764, 0.32350838, 0.32240801,
       0.32020727, 0.32130764, 0.32350838, 0.32460875, 0.32130764,
       0.32350838, 0.32130764, 0.32460875, 0.32350838, 0.32460875,
       0.32350838, 0.32130764, 0.32130764, 0.32240801, 0.32240801,
       0.32350838, 0.32130764, 0.32020727, 0.32350838, 0.32570912,
       0.32130764, 0.32240801, 0.32130764, 0.32130764, 0.32460875,
       0.32460875, 0.32020727, 0.32350838, 0.32020727, 0.32130764,
       0.32020727, 0.32350838, 0.

In [74]:
import pandas as pd
rows = []

for strategy in STRATEGIES:
    for preemption_rate in PREEMPTION_RATES:
        if simulate_results[strategy][preemption_rate] is None:
            continue
        rows.append((strategy, preemption_rate, np.mean(simulate_results[strategy][preemption_rate]), np.percentile(simulate_results[strategy][preemption_rate], 90), np.percentile(simulate_results[strategy][preemption_rate], 95)))

df = pd.DataFrame(rows, columns=['strategy', 'preemption_rate', 'mean', '90th', '95th'])
print(df)


              strategy  preemption_rate        mean        90th        95th
0            only_spot             0.10    0.321310    0.323508    0.323508
1            only_spot             0.20    0.325116    0.327910    0.329010
2            only_spot             0.25    0.327185    0.330111    0.331211
3            only_spot             0.30    0.329365    0.333412    0.333412
4            only_spot             0.35    0.331792    0.335612    0.336713
5            only_spot             0.40    0.334346    0.338914    0.340014
6            only_spot             0.45    0.337174    0.342215    0.343315
7            only_spot             0.50    0.340246    0.345516    0.346616
8            only_spot             0.55    0.343609    0.348817    0.349917
9            only_spot             0.60    0.347389    0.353218    0.355419
10            strawman             0.10    0.322394    0.324609    0.324609
11            strawman             0.20    0.343658    0.388132    0.455341
12          

In [77]:
df[df['strategy'] == 'ideal_ilp_overhead']

,strategy,preemption_rate,mean,90th,95th
30,ideal_ilp_overhead,0.20,0.326506,0.334989,0.343169
31,ideal_ilp_overhead,0.25,0.336928,0.365349,0.377346
32,ideal_ilp_overhead,0.30,0.356212,0.398387,0.412077
33,ideal_ilp_overhead,0.35,0.383786,0.435667,0.447664
34,ideal_ilp_overhead,0.40,0.415575,0.470746,0.483813
35,ideal_ilp_overhead,0.45,0.449285,0.506556,0.520366
36,ideal_ilp_overhead,0.50,0.484550,0.542801,0.559320
37,ideal_ilp_overhead,0.55,0.520297,0.581659,0.597077
38,ideal_ilp_overhead,0.60,0.556251,0.615956,0.630115


In [76]:
print('\n'.join([str(i) for i in df[df['strategy'] == 'ideal_ilp_overhead']['95th'].to_list()]))


0.3431692561684408
0.37734647872989896
0.4120765033810509
0.44766434289978785
0.48381293959336696
0.520365911301082
0.559319831738922
0.5970772553542736
0.6301150010177062


## Real

In [53]:
with open('../exp/on_demand-trace-us-west-2a_v100_1-ddl=1000.0-dur=48.0-over=0.1', 'r') as f:
    on_demand = json.load(f)['costs']

with open('../exp/strawman-trace-us-west-2a_v100_1-ddl=52.0-dur=48.0-over=0.1', 'r') as f:
    strawman = json.load(f)['costs']
    
with open('../exp/pair_amortize_avg_8.0h-trace-us-west-2a_v100_1-ddl=52.0-dur=48.0-over=0.1', 'r') as f:
    pair = json.load(f)['costs']

with open('../exp/ideal_ilp_overhead-trace-us-west-2a_v100_1-ddl=52.0-dur=48.0-over=0.1', 'r') as f:
    ideal = json.load(f)['costs']

with open('../exp/only_spot-trace-us-west-2a_v100_1-ddl=1000.0-dur=48.0-over=0.1', 'r') as f:
    r = json.load(f)
    spot = r['costs']
    spot_time = [t * 10 / 60 / 52 for t in r['ticks']]


results = [spot, strawman, pair, ideal]
funcs = [np.mean, lambda x: np.percentile(x, 90), lambda x: np.percentile(x, 95), np.std]
for result in results:
    result = np.array(result) / np.array(on_demand)
    print([func(result) for func in funcs])

print(np.mean(spot_time), np.std(spot_time), np.percentile(spot_time, 90), np.percentile(spot_time, 95))


[0.3269536333310709, 0.33561243413166786, 0.3367128027681651, 0.005563757953682016]
[0.7436784744555268, 0.9951212203451163, 1.0034602076124566, 0.23336003154338553]
[0.6361778185991813, 0.9442719768414873, 0.9819664269398651, 0.21476060603595545]
[0.4608675726530505, 0.6265090349865413, 0.6533044473844903, 0.10677730917090884]
1.2500865384615385 0.21897885959521746 1.5320512820512822 1.8429807692307691
